In [1]:
# Import the different libraries:

import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import matplotlib as plt
import numpy as np

In [2]:
# Load the DataFrame
exo_nasa = pd.read_csv("../data/combined/main_frame.csv", low_memory=False)    

In [3]:
#Replace NaNs in pl_eqt with temp_calculated from EU table if there is any. Otherwise it keeps the NaN

exo_nasa.pl_eqt = np.where(exo_nasa.pl_eqt.isnull(), exo_nasa.temp_calculated, exo_nasa.pl_eqt)

In [4]:
# Get rid of all NaN temperatures
exo_nasa = exo_nasa.dropna(subset=["pl_eqt"])

In [5]:
looking_at = ["pl_name","st_mass","pl_orbper","st_teff","st_dist","pl_discmethod","pl_bmassj", "pl_eqt","st_rad","pl_dens","star_mass","star_teff","star_distance","detection_type","mass","temp_calculated","star_radius"]
exo_nasa= exo_nasa[looking_at]

In [6]:
# Calculate temperature in Celsius from Kelvin
#exo_nasa['pl_eqt'] = exo_nasa['pl_eqt'] -273

In [7]:
# Inserd a planet ID called pl_id
exo_nasa.insert(0,"pl_id",range(len(exo_nasa)))

In [8]:
# Define all nessessarry functions:
def pa_2_km(st_dist):
    return st_dist * 3.26156 * 365.25 * 24 * 60 * 60 * 300000

def by_car(st_dist_km):
    car_speed = 200 #km/h
    hours_to_year = 24*265.25
    return st_dist_km/200/hours_to_year

def by_plane(st_dist_km):
    plane_speed = 1000 #km/h
    hours_to_year = 24*265.25
    return st_dist_km/plane_speed/hours_to_year

def by_rocket(st_dist_km):
    rocket_speed = 11.2 #km/s
    seconds_to_year = 24*265.25*60*60
    return st_dist_km/rocket_speed/seconds_to_year




In [9]:
exo_nasa["st_dist_km"] = pa_2_km(exo_nasa["st_dist"])

In [10]:
exo_nasa["years_by_car"] = by_car(exo_nasa["st_dist_km"])

In [11]:
exo_nasa["years_by_plane"] = by_plane(exo_nasa["st_dist_km"])

In [12]:
exo_nasa["years_by_rocket"] = by_rocket(exo_nasa["st_dist_km"])

In [13]:
exo_nasa = exo_nasa.sort_values("years_by_rocket")

In [14]:
exo_nasa["pl_mass_e"] = exo_nasa["pl_bmassj"]*317.8

In [15]:
exo_nasa = exo_nasa.drop(columns=["star_teff","star_radius", "star_distance", "star_mass", "st_rad", "detection_type", "mass", "temp_calculated", "pl_dens", "pl_discmethod"])

In [16]:
exo_nasa.to_csv("../data/final/final_frame.csv", index=False)